In [47]:
from prep_data import Data_prepare
import numpy as np 
import os 
# import prep_data

###  Data preprocessing 
1. covert mat data to numpy array 
2. combine male and female data to one array 
3. if needed, remove certain rows 
4. get postive and negative of Xs 
    - order difference or net vs neg 
5. reshape each X: 128*8*2 = 2048 
    - input: data 
6. generate labels for x 
    - input: data and label_type (0:zeros, 1: ones )
7. combine net and neg 
    - input: netX,netY,negX,negY

In [48]:
dp = Data_prepare() 
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
# netural vs postive 
# pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
# neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]] 
m_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat" 
f_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
index=[40,41]
df = dp.combine_male_female(m_file,f_file)
df = dp.remove_person(df,index)
pos_df = dp.get_pos_or_neg(df,pos)
neg_df = dp.get_pos_or_neg(df,neg)
pos_sequeezed = dp.squeeze_feature_size(pos_df)
neg_sequeezed = dp.squeeze_feature_size(neg_df)
pos_labels = dp.generate_labels(pos_sequeezed,1)
neg_labels = dp.generate_labels(neg_sequeezed,0)
x,y = dp.combine_net_neg(pos_sequeezed,pos_labels,neg_sequeezed,neg_labels)

In [49]:
## (40,) -> 40*36*2048
def flatten(data):
    res = data[0]
    for i in range(1,data.shape[0]):
        res = np.append(res,data[i],axis=0)
    return res 
new_x = flatten(x)
new_y = flatten(y)

In [50]:
print(new_x.shape,new_y.shape)


(1404, 2048) (1404, 1)


#### Normalize x with standardScaler

In [51]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(new_x)
new_x = scaler.transform(new_x)

In [52]:
print(new_x.shape,new_y.shape)

(1404, 2048) (1404, 1)


In [53]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, test_size = 0.1, random_state = 4)

In [54]:
print(x_train.shape,y_train.shape)

(1263, 2048) (1263, 1)


In [55]:
## reshape x_train and x_test to 3D for LSTM 
x_train = np.reshape(x_train, (x_train.shape[0],1,new_x.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0],1,new_x.shape[1]))

In [56]:
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [57]:
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(1263, 1, 2048) (141, 1, 2048)
(1263, 1) (141, 1)


#### LSTM model 

In [58]:
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(128, input_shape=(1,2048),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,activation="sigmoid",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16,activation="sigmoid"))
model.add(Dropout(0.2))
# model.add(LSTM(50))
# model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
from keras.optimizers import SGD
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 128)            1114624   
_________________________________________________________________
dropout (Dropout)            (None, 1, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 32)             20608     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 32)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3

In [59]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 13, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/13
 1/40 [..............................] - ETA: 0s - loss: 0.7311 - accuracy: 0.4375WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
40/40 [==============================] - 1s 24ms/step - loss: 0.6954 - accuracy: 0.5083 - val_loss: 0.6760 - val_accuracy: 0.5816
Epoch 2/13
40/40 [==============================] - 0s 9ms/step - loss: 0.6541 - accuracy: 0.6041 - val_loss: 0.6358 - val_accuracy: 0.6738
Epoch 3/13
40/40 [==============================] - 0s 9ms/step - loss: 0.6212 - accuracy: 0.6683 - val_loss: 0.6134 - val_accuracy: 0.6950
Epoch 4/13
40/40 [==============================] - 0s 9ms/step - loss: 0.5867 - accuracy: 0.7086 - val_loss: 0.6062 - val_accuracy: 0.6809
Epoch 5/13
40/40 [==============================] - 0s 9ms/step - loss: 0.5762 

In [60]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [62]:
# %tensorboard --logdir logs

#### CRNN 

In [ ]:
input_shape = (1,2048)
model = Sequential()
model.add(Conv2D(40, (30, 1), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="same", input_shape=input_shape))
model.add(Conv2D(40, (1, nchan), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="valid"))
model.add(AveragePooling2D((30, 1), strides=(15, 1)))
model.add(Flatten())
model.add(Dense(80, activation="relu"))
model.add(Dense(nclasses, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])
return model